# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [1]:
# your code here
url='https://www.zalando.fr/api/catalog/articles?categories=promo-femme&limit=84&offset=84&sort=popularity'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [2]:
# your code here
import re
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
response = requests.get(url)


In [4]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "urllib request",
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "id": "xiPmnzAbH6vl"
      },
      "source": [
        "import re\n",
        "import json\n",
        "import requests\n",
        "import urllib.request"
      ],
      "execution_count": 20,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "O1faNnLfH5eH"
      },
      "source": [
        "url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-femme&limit=84&offset=84&sort=popularity'"
      ],
      "execution_count": 27,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "p9TGPqKnMMSW"
      },
      "source": [
        "request = urllib.request.urlopen(url)\n",
        "data = json.load(request)"
      ],
      "execution_count": 28,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "GXDZZub9Mgvs",
        "outputId": "6c6dd2d2-4793-4405-ae22-8f636aaa78b1",
        "colab": {
          "base_uri": "https://localhost:8080/"
        }
      },
      "source": [
        "data.keys()"
      ],
      "execution_count": 30,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "dict_keys(['total_count', 'pagination', 'sort', 'articles', 'query_path', 'previous_page_path', 'next_page_path', 'query_params', 'page_gender', 'premium', 'appliedSort', 'filters', 'total_article_count', 'single_assortment_entrypoints', 'multi_assortment_entrypoints', 'filter_groups', 'plusStatus', 'categoryTree', 'sortingKeys', 'breadcrumbs', 'querySemantics', 'articlesToShow', 'octopusTests', 'locale', 'isLoggedIn', 'notification', 'resetFilters', 'selectedFilters', 'ssrArticleCount', 'feedbackId', 'variants', 'contentPositions', 'followBrandsBannerEnabled', 'prideEnabled', 'lazyArticleImages', 'hideSearchTerm', 'iconPaths', 'teaser', 'inCatalogTeaser', 'entryPointTeasers', 'upperInCatTeaser', 'inGridTeaser', 'wishlist', 'pills', 'sizeOnboardingDialog'])"
            ]
          },
          "metadata": {
            "tags": []
          },
          "execution_count": 30
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "wkgAkqvEMnly",
        "outputId": "5645786d-f549-45e3-d3ab-5f86b9faf19d",
        "colab": {
          "base_uri": "https://localhost:8080/"
        }
      },
      "source": [
        "data['total_count']"
      ],
      "execution_count": 31,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "193676"
            ]
          },
          "metadata": {
            "tags": []
          },
          "execution_count": 31
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "YH00HxrRNUiz"
      },
      "source": [
        ""
      ],
      "execution_count": 0,
      "outputs": []
    }
  ]
}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'name': 'urllib request', 'provenance': []},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'}},
 'cells': [{'cell_type': 'code',
   'metadata': {'id': 'xiPmnzAbH6vl'},
   'source': ['import re\n',
    'import json\n',
    'import requests\n',
    'import urllib.request'],
   'execution_count': 20,
   'outputs': []},
  {'cell_type': 'code',
   'metadata': {'id': 'O1faNnLfH5eH'},
   'source': ["url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-femme&limit=84&offset=84&sort=popularity'"],
   'execution_count': 27,
   'outputs': []},
  {'cell_type': 'code',
   'metadata': {'id': 'p9TGPqKnMMSW'},
   'source': ['request = urllib.request.urlopen(url)\n',
    'data = json.load(request)'],
   'execution_count': 28,
   'outputs': []},
  {'cell_type': 'code',
   'metadata': {'id': 'GXDZZub9Mgvs',
    'outputId': '6c6dd2d2-4793-4405-ae22-8f636aaa78b1',
    'colab': {'base_uri': 'https://localhost:8080/'}

In [5]:
import urllib.request

In [7]:
request = urllib.request.urlopen(url)
json_url = urllib.request.urlopen(url)
results = json.loads(json_url.read())
results

{'total_count': 190573,
 'pagination': {'page_count': 892, 'current_page': 2, 'per_page': 84},
 'sort': 'popularity',
 'articles': [{'sku': 'M9121I1SA-B11',
   'name': 'SAYN - Pullover - écru',
   'price': {'original': '29,99\xa0€',
    'promotional': '23,99\xa0€',
    'has_different_prices': False,
    'has_different_original_prices': False,
    'has_different_promotional_prices': False,
    'has_discount_on_selected_sizes_only': False},
   'sizes': ['S', 'L'],
   'url_key': 'mango-pullover-ecru-m9121i1sa-b11',
   'media': [{'path': 'spp-media-p1/56133ba8dffa341ebe59d67ffc35c165/389a6a6b69ac4732bd6b8dc2ae801347.jpg',
     'role': 'DEFAULT',
     'packet_shot': False},
    {'path': 'spp-media-p1/f3368368186f372a80e8929fcf8d3981/5994cdcd10b94291893b30c70641e977.jpg',
     'role': 'HOVER',
     'packet_shot': False}],
   'brand_name': 'Mango',
   'is_premium': False,
   'family_articles': [],
   'flags': [{'key': 'discountRate',
     'value': '-20%',
     'tracking_value': 'discount rate

In [9]:
flat_data = json_normalize(results)

df = json_normalize(flat_data.articles[0])
df.head()

<ipython-input-9-22aa60b96648>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flat_data = json_normalize(results)
<ipython-input-9-22aa60b96648>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(flat_data.articles[0])


,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,price.base_price,outfits
0,M9121I1SA-B11,SAYN - Pullover - écru,"[S, L]",mango-pullover-ecru-m9121i1sa-b11,[{'path': 'spp-media-p1/56133ba8dffa341ebe59d6...,Mango,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"29,99 €","23,99 €",False,False,False,False,NaN,NaN,NaN
1,UG111Y00A-O11,CLASSIC MINI II - Bottines - chestnut,"[36, 37, 38, 39, 40, 41, 42, 43]",ugg-classic-mini-ii-bottines-chestnut-ug111y00...,[{'path': 'spp-media-p1/cc82b8d35bc53548828ec3...,UGG,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -14...",shoe,[],"169,95 €","146,95 €",True,False,True,False,NaN,NaN,NaN
2,BU311A0CE-Q11,CHAI - Baskets basses - black,"[37, 38, 39, 40, 41]",buffalo-chai-reproduction-baskets-basses-black...,[{'path': 'spp-media-p1/43c8cd5fc42a304db34fe1...,Buffalo,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",shoe,[],"99,99 €","74,99 €",False,False,False,False,NaN,NaN,NaN
3,LE222S03D-C11,NEW ORIGINAL CREW UNISEX - Sweatshirt - chisel...,"[S, M, L, XL, XXL]",levis-sweatshirt-chisel-grey-heather-le222s03d...,[{'path': 'spp-media-p1/153909cb58f63af49a574f...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",clothing,[],"64,95 €","48,45 €",False,False,False,False,NaN,NaN,NaN
4,AD122E08B-Q11,UNISEX - Pantalon de survêtement - black/white,"[XS, S, M, L, XL, XXL]",adidas-originals-pantalon-de-survetement-black...,[{'path': 'spp-media-p1/c83c873b01c93adf8cbcf0...,adidas Originals,False,[],"[{'key': 'discountRate', 'value': '-40%', 'tra...",clothing,[],"59,95 €","35,95 €",False,False,False,False,NaN,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# your code here